In [13]:
import numpy as np
from numpy import log
import pandas as pd
from pandas import *

from datetime import datetime, date

from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import scipy.stats as stats
import glob

In [14]:
# Read in the regression input data
df = pd.read_csv(r"C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\R Regression\otto_regression_input_no_neg_pysent.csv", index_col =0 )

# Read in sport pe data
sports_pe = pd.read_csv(r"C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\Data\Otto data\Sports\nyc_metro_sports_2012.csv", index_col = 0)

In [15]:
list(df.columns)

['ZIP',
 'z_ses',
 'county',
 'purchase',
 'log_purchase',
 'HURRICANE',
 'HOLIDAYS',
 'NEWYEARSDAY',
 'DAYAFTERNEWYEARS',
 'BIRTHDAYOFMARTINLUTHERKINGJR',
 'VALENTINESDAY',
 'WASHINGTONSBIRTHDAY',
 'EASTER',
 'MEMORIALDAY',
 'INDEPENDENCEDAY',
 'LABORDAY',
 'COLUMBUSDAY',
 'VETERANSDAY',
 'THANKSGIVING',
 'CHRISTMASDAY',
 'DAYAFTERCHRISTMAS',
 'NEWYEARSEVE',
 'FIRST_OF_MONTH',
 'FIFTEENTH_OF_MONTH',
 'MON',
 'TUE',
 'WED',
 'THU',
 'FRI',
 'SAT',
 'SUN',
 'JAN',
 'FEB',
 'MAR',
 'APR',
 'MAY',
 'JUN',
 'JUL',
 'AUG',
 'SEP',
 'OCT',
 'NOV',
 'DEC ',
 'devils_vader_pe',
 'giants_vader_pe',
 'knicks_vader_pe',
 'mets_vader_pe',
 'yankees_vader_pe',
 'devils_pysent_pe',
 'giants_pysent_pe',
 'knicks_pysent_pe',
 'mets_pysent_pe',
 'yankees_pysent_pe',
 'devils_mean_no_neg_pysent',
 'giants_mean_no_neg_pysent',
 'knicks_mean_no_neg_pysent',
 'mets_mean_no_neg_pysent',
 'yankees_mean_no_neg_pysent',
 'devils_mean_roberta',
 'giants_mean_roberta',
 'knicks_mean_roberta',
 'mets_mean_roberta

In [16]:
DATE_RANGES = pd.date_range(datetime(2012, 1, 1), datetime(2012, 12,31))

### Replace sent scores with means on days when the team has less than 5 tweets

In [17]:
# Create a dictionray with the team name and date where there are less than 5 tweets
list_of_files = glob.glob(r"C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\Data\Preprocessed tweets\*.csv")
list_of_files

['C:\\Users\\clark\\OneDrive - University of Warwick\\Diss\\2. Analysis\\Data\\Preprocessed tweets\\2012_devils_processed.csv',
 'C:\\Users\\clark\\OneDrive - University of Warwick\\Diss\\2. Analysis\\Data\\Preprocessed tweets\\2012_giants_processed.csv',
 'C:\\Users\\clark\\OneDrive - University of Warwick\\Diss\\2. Analysis\\Data\\Preprocessed tweets\\2012_knicks_processed.csv',
 'C:\\Users\\clark\\OneDrive - University of Warwick\\Diss\\2. Analysis\\Data\\Preprocessed tweets\\2012_mets_processed.csv',
 'C:\\Users\\clark\\OneDrive - University of Warwick\\Diss\\2. Analysis\\Data\\Preprocessed tweets\\2012_TEST_processed.csv',
 'C:\\Users\\clark\\OneDrive - University of Warwick\\Diss\\2. Analysis\\Data\\Preprocessed tweets\\2012_yankees_processed.csv']

In [18]:
# This function takes a file path as input and outputs a dataframe with dates and tweet counts less than 5
def tweets_per_day(data):
    
    # Load in data 
    df = pd.read_csv(data)
    # Date ranges
    DATE_RANGES = pd.date_range(date(2012, 1, 1), date(2012, 12,31))

    # Update this with the count of tweets each day
    tweets_per_day = pd.DataFrame(columns = ["num_tweets"])
    
    df["created_at"] = pd.to_datetime(df["created_at"]).dt.date
    
    group_df = df.groupby("created_at")
    
    for d, day in group_df:
        
        # How many tweets are there on this day?
        num_tweets = len(day)
        # Update tweets per day
        #print(d)
        tweets_per_day.at[d, "num_tweets"] = num_tweets
    
    # Select the days where there are less than 5 tweets
    less_than_five = tweets_per_day[tweets_per_day["num_tweets"] <= 5]
    
    #print(less_than_five)
    return less_than_five

In [19]:
# A dictionary with the team names and respective file paths
dict_of_files = {"devils": list_of_files[0],
                "giants": list_of_files[1],
                "knicks": list_of_files[2],
                "mets": list_of_files[3],
                "yankees": list_of_files[4]
                }

# Populate this dictionary with the dates upon which there are fewer than 5 tweets
less_than_5_days = {}

# Loop through each team, find dates with less than 5 and add to less_than_5_days dictionary
for key, file in dict_of_files.items():
    # Call tweets_per_day
    l_t_5= tweets_per_day(file)
    
    # Add all the dates to the less_than_5_days dict
    less_than_5_days[key] = [date_obj.strftime("%Y-%m-%d") for date_obj in list(l_t_5.index)]
    
        
less_than_5_days 

{'devils': ['2012-10-30',
  '2012-11-04',
  '2012-11-13',
  '2012-11-23',
  '2012-12-18',
  '2012-12-24'],
 'giants': [],
 'knicks': ['2012-06-01',
  '2012-06-03',
  '2012-06-09',
  '2012-06-11',
  '2012-07-30',
  '2012-08-17',
  '2012-09-01',
  '2012-09-04',
  '2012-09-15',
  '2012-09-19',
  '2012-09-23'],
 'mets': [],
 'yankees': ['2012-01-01']}

In [20]:
df.columns

Index(['ZIP', 'z_ses', 'county', 'purchase', 'log_purchase', 'HURRICANE',
       'HOLIDAYS', 'NEWYEARSDAY', 'DAYAFTERNEWYEARS',
       'BIRTHDAYOFMARTINLUTHERKINGJR', 'VALENTINESDAY', 'WASHINGTONSBIRTHDAY',
       'EASTER', 'MEMORIALDAY', 'INDEPENDENCEDAY', 'LABORDAY', 'COLUMBUSDAY',
       'VETERANSDAY', 'THANKSGIVING', 'CHRISTMASDAY', 'DAYAFTERCHRISTMAS',
       'NEWYEARSEVE', 'FIRST_OF_MONTH', 'FIFTEENTH_OF_MONTH', 'MON', 'TUE',
       'WED', 'THU', 'FRI', 'SAT', 'SUN', 'JAN', 'FEB', 'MAR', 'APR', 'MAY',
       'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC ', 'devils_vader_pe',
       'giants_vader_pe', 'knicks_vader_pe', 'mets_vader_pe',
       'yankees_vader_pe', 'devils_pysent_pe', 'giants_pysent_pe',
       'knicks_pysent_pe', 'mets_pysent_pe', 'yankees_pysent_pe',
       'devils_mean_no_neg_pysent', 'giants_mean_no_neg_pysent',
       'knicks_mean_no_neg_pysent', 'mets_mean_no_neg_pysent',
       'yankees_mean_no_neg_pysent', 'devils_mean_roberta',
       'giants_mean_roberta'

In [21]:
# Columns that we want to replace 
replace_cols = [""]

# for each team
for team, dates in less_than_5_days.items():
    
    # select all rows not have less than 5
    more_than_5 = df[~df.index.isin(dates)]
    
    # Get a list of the sentiment scores relating to that team
    cols_list = list(df.columns[df.columns.str.contains(team)])
    
    for col in cols_list:
        # Get the mean of the more than 5 
        mean_ = more_than_5[col].mean()
#         print("The mean for ", team, " ", col, "is ", mean_)
        # Replace values in original df with mean_
        df.loc[df.index.isin(dates),col]= mean_
        


#### Aggregate the to find the mean sales, sentiment/surprise scores for each day
#### Create a new dataframe 

In [25]:
new_df = pd.DataFrame()

# Iterate through each date
date_grouped = df.groupby(df.index)

for d, date_df in date_grouped:
    
    # Access first row of date_df
    first_row = date_df.iloc[[0]]
    
    # Assign first row 
    new_df = pd.concat([new_df, first_row])
    
    # Update the purchase figure
    # Find the mean of the composite_per_capita sales
    date_mean_sales = np.mean(date_df["purchase"])
    new_df.loc[d,"purchase"] = date_mean_sales

# drop ZIP, z_ses, county and log_purchase
new_df = new_df.drop(["ZIP","z_ses", "county", "log_purchase"], axis = 1)

# Add a log column - natural logarithm 
new_df['log_purchase'] = log(new_df["purchase"])

# Sum together the vader, pysent and roberta scores
vader_cols = list(new_df.columns[new_df.columns.str.contains("vader")])
sum_vader = new_df[vader_cols].sum(axis = 1)
new_df["sum_vader"] = sum_vader

pysent_cols = list(new_df.columns[new_df.columns.str.contains("pysent")])
sum_pysent = new_df[pysent_cols].sum(axis = 1)
new_df["sum_pysent"] = sum_pysent

rob_cols = list(new_df.columns[new_df.columns.str.contains("roberta")])
sum_rob = new_df[rob_cols].sum(axis = 1)
new_df["sum_roberta"] = sum_rob

distilbert_cols = list(new_df.columns[new_df.columns.str.contains("distilbert")])
sum_distilbert = new_df[distilbert_cols].sum(axis = 1)
new_df["sum_distilbert"] = sum_distilbert

no_neg_cols = list(new_df.columns[new_df.columns.str.contains("no_neg")])
sum_no_neg = new_df[no_neg_cols].sum(axis = 1)
new_df["sum_no_neg_pysent"] = sum_no_neg


# Create the next day purchases data and replace the NaN value on the last day with mean sales
new_df["next_day_purchase"] = new_df["purchase"].shift(-1)
new_df["next_day_purchase"].fillna(new_df["next_day_purchase"].mean())

# Add the log of next day purchases
new_df["next_day_log_purchase"] = log(new_df["next_day_purchase"])

# Add in the sports prediction errors
new_df["sum_pe"] = list(sports_pe["sum_pe"])
new_df["knicks_pe"] = list(sports_pe["knicks_pe"])
new_df["mets_pe"] = list(sports_pe["mets_pe"])
new_df["yankees_pe"] = list(sports_pe["yankees_pe"])
new_df["giants_pe"] = list(sports_pe["giants_pe"])
new_df["devils_pe"] = list(sports_pe["devils_pe"])

# Normalise the sentiment/surprise scores

# scaler = MinMaxScaler()
# scaler = StandardScaler()
# new_df["norm_sum_vader"] = scaler.fit_transform(np.array(new_df["sum_vader"]).reshape(-1,1))
# new_df["norm_sum_pysent"] = scaler.fit_transform(np.array(new_df["sum_pysent"]).reshape(-1,1))
# new_df["norm_sum_roberta"] = scaler.fit_transform(np.array(new_df["sum_roberta"]).reshape(-1,1))

new_df["norm_sum_vader"] = stats.zscore(new_df["sum_vader"])
new_df["norm_sum_pysent"] = stats.zscore(new_df["sum_pysent"])
new_df["norm_sum_roberta"] = stats.zscore(new_df["sum_roberta"])
new_df["norm_sum_distilbert"] = stats.zscore(new_df["sum_distilbert"])
new_df["norm_sum_no_neg_pysent"] = stats.zscore(new_df["sum_no_neg_pysent"])

In [26]:
# Output the new dataframe
new_df.to_csv(r"C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\R Regression\aggregate_regression_input_7.csv")

In [27]:
# Get the rows where date is sum_pe is zero

non_zero_pe = new_df[new_df["sum_pe"] != 0]

non_zero_pe.to_csv(r"C:\Users\clark\OneDrive - University of Warwick\Diss\2. Analysis\R Regression\aggregate_regression_input_no_zero_pe.csv")
